In [286]:
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [287]:
data = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')

In [288]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [289]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [290]:
data.isnull().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType        872
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [291]:
drop_need = data.loc[:,data.isnull().mean() > 0.2]

In [292]:

data = data.drop(drop_need,axis=1)

In [293]:
need_check = data.loc[:,data.isna().mean() > 0.00001].head()
{col : data[col].unique() for col in need_check}

{'LotFrontage': array([ 65.,  80.,  68.,  60.,  84.,  85.,  75.,  nan,  51.,  50.,  70.,
         91.,  72.,  66., 101.,  57.,  44., 110.,  98.,  47., 108., 112.,
         74., 115.,  61.,  48.,  33.,  52., 100.,  24.,  89.,  63.,  76.,
         81.,  95.,  69.,  21.,  32.,  78., 121., 122.,  40., 105.,  73.,
         77.,  64.,  94.,  34.,  90.,  55.,  88.,  82.,  71., 120., 107.,
         92., 134.,  62.,  86., 141.,  97.,  54.,  41.,  79., 174.,  99.,
         67.,  83.,  43., 103.,  93.,  30., 129., 140.,  35.,  37., 118.,
         87., 116., 150., 111.,  49.,  96.,  59.,  36.,  56., 102.,  58.,
         38., 109., 130.,  53., 137.,  45., 106., 104.,  42.,  39., 144.,
        114., 128., 149., 313., 168., 182., 138., 160., 152., 124., 153.,
         46.]),
 'MasVnrArea': array([1.960e+02, 0.000e+00, 1.620e+02, 3.500e+02, 1.860e+02, 2.400e+02,
        2.860e+02, 3.060e+02, 2.120e+02, 1.800e+02, 3.800e+02, 2.810e+02,
        6.400e+02, 2.000e+02, 2.460e+02, 1.320e+02, 6.500e+02, 1.01

In [294]:
data['LotFrontage'] = data['LotFrontage'].fillna(data['LotFrontage'].mean())
data['MasVnrArea'] = data['MasVnrArea'].fillna(data['MasVnrArea'].mean())

In [295]:
data = data.dropna(axis=0)

In [296]:
{cols : list(data[cols].unique()) for cols in data.columns if data[cols].dtypes == 'object'}

{'MSZoning': ['RL', 'RM', 'C (all)', 'FV', 'RH'],
 'Street': ['Pave', 'Grvl'],
 'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
 'LandContour': ['Lvl', 'Bnk', 'Low', 'HLS'],
 'Utilities': ['AllPub', 'NoSeWa'],
 'LotConfig': ['Inside', 'FR2', 'Corner', 'CulDSac', 'FR3'],
 'LandSlope': ['Gtl', 'Mod', 'Sev'],
 'Neighborhood': ['CollgCr',
  'Veenker',
  'Crawfor',
  'NoRidge',
  'Mitchel',
  'Somerst',
  'NWAmes',
  'OldTown',
  'BrkSide',
  'Sawyer',
  'NridgHt',
  'NAmes',
  'SawyerW',
  'IDOTRR',
  'MeadowV',
  'Timber',
  'Gilbert',
  'StoneBr',
  'ClearCr',
  'Edwards',
  'NPkVill',
  'Blmngtn',
  'BrDale',
  'SWISU',
  'Blueste'],
 'Condition1': ['Norm',
  'Feedr',
  'PosN',
  'Artery',
  'RRAe',
  'RRNn',
  'RRAn',
  'PosA',
  'RRNe'],
 'Condition2': ['Norm',
  'Artery',
  'RRNn',
  'Feedr',
  'PosN',
  'PosA',
  'RRAn',
  'RRAe'],
 'BldgType': ['1Fam', '2fmCon', 'TwnhsE', 'Duplex', 'Twnhs'],
 'HouseStyle': ['2Story',
  '1Story',
  '1.5Fin',
  '1.5Unf',
  'SFoyer',
  'SLvl',
  '2.5Unf',
 

In [297]:
cols_nominal = ['Neighborhood','RoofStyle','RoofMatl','Exterior1st','Exterior2nd',
               'MSZoning','LandSlope','LotConfig','SaleCondition','SaleType',
               'Condition1','Condition2','BldgType','HouseStyle','Street','LotShape',
                'LandContour','Utilities','ExterQual','ExterCond','Foundation','BsmtQual',
                'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC',
                'CentralAir','Electrical','KitchenQual','Functional','GarageType','GarageFinish',
                'GarageQual','GarageCond','PavedDrive'
               ]
               
prefixes =['NEI_','ROOFST_','ROOFMAT','EX1_','EX2_','MSZ_','LANDS_','LOTC_','SALEC_','SaleT_',
          'C1','C2','BLD','HST','ST','LOTSH','LANDC','UTIL','EXQ','EXC','FD','BQ','BC','BX','BF',
          'BF2','HEAT','HEATQ','CAIR','ELEC','KQ','FUNC','GTY','GFS','GQL','GCD','PD']

In [298]:
for col , prefix in zip(cols_nominal , prefixes):
    dummie = pd.get_dummies(data[col],prefix=prefix).astype(np.int)
    data = pd.concat([data,dummie],axis=1)
    data = data.drop(col,axis=1)

/tmp/ipykernel_32/4129719837.py:2: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/tmp/ipykernel_32/4129719837.py:2: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://

In [299]:
data['SalePrice'] =  pd.qcut(data['SalePrice'] , q=2,labels =[0,1]).astype(np.int)

/tmp/ipykernel_32/1227126814.py:1: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [300]:
y = data['SalePrice']
x = data.drop('SalePrice',axis=1)

In [301]:
#x.shape
y.mean()

0.49925705794947994

In [302]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [303]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.7,random_state=7)

In [309]:
inputs = tf.keras.Input(shape=(262,))
x = tf.keras.layers.Dense(64,activation='relu')(inputs)
x = tf.keras.layers.Dense(64,activation='relu')(x)

outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs = inputs , outputs = outputs)

model.compile(
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

hist = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size=40,
    epochs=5
)

Epoch 1/5
19/19 [==============================] - 1s 13ms/step - loss: 0.4314 - accuracy: 0.8167 - val_loss: 0.3460 - val_accuracy: 0.8571
Epoch 2/5
19/19 [==============================] - 0s 4ms/step - loss: 0.2138 - accuracy: 0.9190 - val_loss: 0.2975 - val_accuracy: 0.8836
Epoch 3/5
19/19 [==============================] - 0s 4ms/step - loss: 0.1433 - accuracy: 0.9602 - val_loss: 0.2786 - val_accuracy: 0.8995
Epoch 4/5
19/19 [==============================] - 0s 5ms/step - loss: 0.1026 - accuracy: 0.9655 - val_loss: 0.2606 - val_accuracy: 0.8889
Epoch 5/5
19/19 [==============================] - 0s 5ms/step - loss: 0.0703 - accuracy: 0.9761 - val_loss: 0.2763 - val_accuracy: 0.9101


In [310]:
fig = px.line(
    hist.history,
    y=['loss','val_loss']
)
fig.show()

In [311]:
np.argmin(hist.history['val_loss']) + 1

4

In [312]:
model.evaluate(x_test,y_test)

13/13 [==============================] - 0s 2ms/step - loss: 0.2167 - accuracy: 0.9134


[0.2167065292596817, 0.9133663177490234]

In [308]:
model = LogisticRegression()
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.8935643564356436

url : https://www.kaggle.com/datasets/dansbecker/home-data-for-ml-course        
downlaod : https://www.kaggle.com/datasets/dansbecker/home-data-for-ml-course/download?datasetVersionNumber=1